In [189]:
%display latex

def frac(x):
    return x - floor(RR(x))

def pivot(xs, *indices):
    coeffs = []
    for l in indices:
        a = [0] * len(xs)
        ys = [0] * len(xs)
        xl = xs[l]
        assert xl != 0, "xl cannot be zero"
        for i, xi in enumerate(xs):
            a[i] = floor(RR(xi))
            if i == l:
                ys[i] = 1 / frac(xi)
            else:
                ys[i] = frac(xi) / frac(xl)
        xs = tuple(ys)
        coeffs.append(tuple(a))
    return xs, coeffs

def argmin(xs):
    j = None
    for i in range(len(xs)):
        if j == None or xs[i] < xs[j]:
            j = i
    return j

def pivot_table(xs, pivots):
    ys = xs
    rows = []
    for l in pivots:
        zs, coeffs = pivot(ys, l)
        rows.append((l, *ys, *[round(RR(y), 5) for y in ys], *coeffs[-1]))
        ys = zs

    header_row = [r'$\ell$', '$x_1$', '$x_2$', '$x_1$', '$x_2$', '$a_1$', '$a_2$']
    return table(rows, header_row=header_row)

The modified update rule does not use a negative for the other indices.
$$
  x_i = 
  \begin{cases}
    \frac{1}{x_i}, & \text{ if } i = \ell, \\
    \frac{x_i}{x_\ell}, & \text{ otherwise.} \\
  \end{cases}
$$

# Brute-Force Search

The brute-force search is done in a breadth-first fashion.
We try every list of length $n$ and then every list of length $n + 1$,
until we find a period in the representation.

In [190]:
def brute_force(xs, max_depth):
    d = len(xs)
    queue = [([xs], [])]
    for depth in range(max_depth + 1):
        next_queue = []
        while queue:
            inputs, pivots = queue.pop(0)
            ys = inputs[-1]

            for l in range(d):
                if frac(ys[l]) != 0:
                    zs, _ = pivot(ys, l)
                    try:
                        repeat_start = inputs.index(zs)
                        init = pivots[:repeat_start]
                        repeat = pivots[repeat_start:] + [l]
                        return init, repeat
                    except ValueError:
                        elem = (inputs + [zs], pivots + [l])
                        next_queue.append(elem)
        queue = next_queue
    return []

def format_period(init, repeat):
    init_str = ', '.join([str(e) for e in init])
    repeat_str = ', '.join([str(e) for e in repeat])
    return LatexExpr(rf'[{init_str}, \overline{{{repeat_str}}}]')

def periodic_pivot_table(xs, init, repeat):
    return pivot_table(xs, init + repeat + [repeat[0]])

Here is, for example, the period for $\sqrt[3]{4}$. Notably, this value likely doesn't have a periodic representation in the Jacobi-Perron algorithm. First, the pivots which lead to a period are shown. The period is marked with a line over the pivots. The actual representation of $\sqrt[3]{4}$ is shown in the next block.

In [191]:
p = x^3 - 4
K.<psi> = NumberField(p, embedding=RR(1))

xs = tuple(psi^i for i in range(1,d))
init, repeat = brute_force(xs, 20)
format_period(init, repeat)

[0, \overline{0, 1, 0, 0, 1, 1, 0, 0}]

In [194]:
periodic_pivot_table(xs, init, repeat)

\(\ell\),\(x_1\),\(x_2\),\(x_1\),\(x_2\),\(a_1\),\(a_2\)
\(0\),\(\psi\),\(\psi^{2}\),\(1.5874\),\(2.51984\),\(1\),\(2\)
\(0\),\(\frac{1}{3} \psi^{2} + \frac{1}{3} \psi + \frac{1}{3}\),\(-\frac{1}{3} \psi^{2} + \frac{2}{3} \psi + \frac{2}{3}\),\(1.70241\),\(0.88499\),\(1\),\(0\)
\(1\),\(\frac{1}{4} \psi^{2} + \frac{1}{2} \psi\),\(\frac{1}{2} \psi^{2}\),\(1.42366\),\(1.25992\),\(1\),\(1\)
\(0\),\(\frac{1}{4} \psi^{2} + 1\),\(\frac{1}{2} \psi^{2} + \psi + 1\),\(1.62996\),\(3.84732\),\(1\),\(3\)
\(0\),\(\psi\),\(\psi^{2} - 2 \psi + 2\),\(1.5874\),\(1.34504\),\(1\),\(1\)
\(1\),\(\frac{1}{3} \psi^{2} + \frac{1}{3} \psi + \frac{1}{3}\),\(\psi - 1\),\(1.70241\),\(0.5874\),\(1\),\(0\)
\(1\),\(\frac{1}{3} \psi + \frac{2}{3}\),\(\frac{1}{3} \psi^{2} + \frac{1}{3} \psi + \frac{1}{3}\),\(1.1958\),\(1.70241\),\(1\),\(1\)
\(0\),\(\frac{1}{12} \psi^{2} - \frac{1}{6} \psi + \frac{1}{3}\),\(\frac{1}{4} \psi^{2} + \frac{1}{2} \psi\),\(0.27875\),\(1.42366\),\(0\),\(1\)
\(0\),\(\psi + 2\),\(\psi^{2} - 1\),\(3.5874\),\(1.51984\),\(3\),\(1\)
\(0\),\(\frac{1}{3} \psi^{2} + \frac{1}{3} \psi + \frac{1}{3}\),\(-\frac{1}{3} \psi^{2} + \frac{2}{3} \psi + \frac{2}{3}\),\(1.70241\),\(0.88499\),\(1\),\(0\)


# Equal Distance between Neighbors

The first polynomial is the "neighbor" polynomial, which is derived from the following equation:
$$
  x_1 = \frac{x_2}{x_1} - 1 = \dots = \frac{x_d}{x_{d-1}} - 1 = \frac{1}{x_d} - 1.
$$
This results in the equation
$$
  x_d^{d+1} + x_d - 1 = 0, \quad x_i = x_d^{d + 1 - i}
$$

In [159]:
def neighbor_distance(xs):
    dist = []
    for x, y in zip([*xs, 1], [1, *xs]):
        if y == 0 or x == 0:
            dist.append(1)
        else:
            dist.append(frac(x / y))
    return dist

def distance_matrix(xs):
    D = [[0 for _ in xs] for _ in xs]
    for i, x in enumerate(xs):
        for j, y in enumerate(xs):
            if x == 0 or y == 0:
                D[i][j] = 1
            elif i == j:
                D[i][j] = frac(1 / x)
            else:
                D[i][j] = frac(x / y)
    return D

# Indices are zero-based. An index of -1 indicates selecting nothing in the first iteration and selecting 0 in the next iteration.
# Similarly, d - 1 indicates selecting d - 1 in the first iteration and nothing in the next iteration.
def select(xs):
    min_index = None
    min_dist = 1
    for i, dist in enumerate(neighbor_distance(xs)):
        if min_dist > dist:
            min_dist = dist
            min_index = i - 1
    return min_index

In [160]:
def neighbor_poly(d):
    return x ** (d+1) + x - 1
neighbor_poly(5)

x^6 + x - 1

The polynomial gives us the field $\mathbb{Q}/ (x^{d+1} + x - 1)$.

In [181]:
d = 3
p = neighbor_poly(d)
K.<psi> = NumberField(p, embedding=RR(0.5))
K

Number Field in psi with defining polynomial x^4 + x - 1 with psi = 0.7244919590005156?

For our input, we choose $x_i = \psi^{d+1-i}$.

In [182]:
xs = tuple(sorted([psi^(d+1-i) for i in range(1,d+1)]))

table([(i, x,RR(x)) for i, x in enumerate(xs)], header_row=[r'$i$', r'$x_i$'])

$i$   $x_i$
├─────┼───────┤
  0     psi^3
  1     psi^2
  2     psi

The distance between neighbors is indeed the same.

In [183]:
dist = neighbor_distance(xs)

table([dist, [round(RR(d), 3) for d in dist]])

\(\psi^{3}\),\(\psi^{3}\),\(\psi^{3}\),\(\psi^{3}\)
\(0.38\),\(0.38\),\(0.38\),\(0.38\)


Therefore, we can expect a decrease of at least $\psi^3$ over two iterations. However, this condition does not hold up after those two iterations when choosing any element to pivot with. It only holds up if we choose the maximum $x_d$ of those elements. Choosing any other pivot, destroys this solution.

In [188]:
n = 10
d = len(xs)

ys = xs
rows = [[
    *[f'$x_{i+1}$' for i in range(d)], 
    *[fr'$\{{x_{{{i+2}}}/x_{{{i+1}}}\}}$' for i in range(-1, d)], 
    r'$\ell_1$', 
    r'$\ell_2$',
]]

for i in range(n):
    # select pivot
    l = select(ys)
    
    # Add row to the output
    dist   = [round(RR(d), 3) for d in neighbor_distance(ys)]
    values = [round(RR(y), 3) for y in ys]
    rows.append([*values, *dist, '-' if l == -1 else l, '-' if l == d - 1 else l + 1])

    # pivot
    if l == -1:
        ys, _ = pivot(ys, 0)
    elif l == d - 1:
        ys, _ = pivot(ys, d - 1)
    else:
        ys, _ = pivot(ys, l, l + 1)
    
table(rows, header_row=True, align='center')

\(x_1\),\(x_2\),\(x_3\),\(\{x_{1}/x_{0}\}\),\(\{x_{2}/x_{1}\}\),\(\{x_{3}/x_{2}\}\),\(\{x_{4}/x_{3}\}\),\(\ell_1\),\(\ell_2\)
\(0.38\),\(0.525\),\(0.724\),\(0.38\),\(0.38\),\(0.38\),\(0.38\),-,\(0\)
\(0.63\),\(0.38\),\(0.905\),\(0.63\),\(0.604\),\(0.38\),\(0.105\),\(2\),-
\(0.696\),\(0.42\),\(0.105\),\(0.696\),\(0.604\),\(0.249\),\(0.545\),\(1\),\(2\)
\(0.63\),\(0.525\),\(0.01\),\(0.63\),\(0.834\),\(0.019\),\(0.647\),\(1\),\(2\)
\(0.545\),\(0.819\),\(0.829\),\(0.545\),\(0.503\),\(0.012\),\(0.207\),\(1\),\(2\)
\(0.829\),\(0.254\),\(0.393\),\(0.829\),\(0.307\),\(0.548\),\(0.543\),\(0\),\(1\)
\(0.675\),\(0.262\),\(0.548\),\(0.675\),\(0.388\),\(0.091\),\(0.825\),\(1\),\(2\)
\(0.307\),\(0.942\),\(0.959\),\(0.307\),\(0.073\),\(0.017\),\(0.043\),\(1\),\(2\)
\(0.838\),\(0.536\),\(0.897\),\(0.838\),\(0.64\),\(0.673\),\(0.114\),\(2\),-
\(0.934\),\(0.598\),\(0.114\),\(0.934\),\(0.64\),\(0.192\),\(0.736\),\(1\),\(2\)


## Other polynomials

$$
    x^{d+1} + \sum_{k=1}^d k x^{d-k} = 1
$$

In [9]:
xs = (1, 4^(1/3), 16^(1/3))
indices = [1, 2] + [2]*20
table([[RR(x) for x in pivot(xs, *indices[:i+1])] for i in range(len(indices))])

TypeError: unable to convert '(1/4*4^(2/3),1/4*4^(2/3),1/2*4^(2/3)*2^(1/3)-1)' to a real number

# $d$-bonacci Numbers


In [145]:
def nbonacci_poly(d):
    i = var('i')
    return x^(d+1) + sum(x^i, i, 1, d) - 1
nbonacci_poly(3), *[r[0] for r in nbonacci_poly(1).roots()]

(x^4 + x^3 + x^2 + x - 1, -1/2*sqrt(5) - 1/2, 1/2*sqrt(5) - 1/2)

In [242]:
y = polygen(RR, 'y')
d = 5

p = nbonacci_poly(d)
psi_real = find_root(p, 0, 1)
K.<psi> = NumberField(p, embedding=RR(psi_real))

In [168]:
xs = tuple(sum(psi^i for i in range(1, k+1)) for k in range(1, d + 1))
xs

(psi,
 psi^2 + psi,
 psi^3 + psi^2 + psi,
 psi^4 + psi^3 + psi^2 + psi,
 psi^5 + psi^4 + psi^3 + psi^2 + psi)

In [216]:
ys = pivot(xs, 0)
argmin(ys), ys

(1,
 (psi^5 + psi^4 + psi^3 + psi^2 + psi,
  psi,
  psi^2 + psi,
  psi^3 + psi^2 + psi,
  psi^4 + psi^3 + psi^2 + psi))

In [254]:
q = factor(-x^6 * p(x=1/x))
phi_real = find_root(q, 1, 2)
L.<phi> = NumberField(q, embedding=RR(phi_real))
q.list()

[-1, -1, -1, -1, -1, -1, 1]

# Worst-Case Analysis

In [60]:
d = 3
xs = tuple(i / (d + 1) for i in range(1, d + 1))
pivot(xs, 0)

(0, 0, 0)

In [133]:
def unpivot(y, values, indices):
    for a, l in zip(values, indices):
        x = [0] * len(ys)
        for i, (yi, ai) in enumerate(zip(y, a)):
            if i == l:
                x[i] = frac(1 / (ai + yi))
            else:
                x[i] = frac((ai + yi) / (a[l] + y[l]))
        y = x
    return y

In [165]:
ys = tuple(sorted(unpivot(xs, [(2, 1, 1)] * 5, [0] * 5)))
table(distance_matrix(ys), header_row=ys, header_column=['', *ys])

,\(\frac{128}{309}\),\(\frac{218}{309}\),\(\frac{73}{103}\)
\(\frac{128}{309}\),\(\frac{53}{128}\),\(\frac{64}{109}\),\(\frac{128}{219}\)
\(\frac{218}{309}\),\(\frac{45}{64}\),\(\frac{91}{218}\),\(\frac{218}{219}\)
\(\frac{73}{103}\),\(\frac{91}{128}\),\(\frac{1}{218}\),\(\frac{30}{73}\)


In [163]:
pivot(ys, 0, 1, 2)

(0, 0, 0)

In [16]:
K.<phi> = NumberField(x^3 - x - 1, embedding=RR(2))
psi = 1/phi

A = matrix([[1, phi, phi^2], [1, -psi]]).transpose()
b = vector([1, 1])
xs = A.solve_right(b)
pivot(xs, 1, 1, *[1]*10)

(0, phi - 1)

In [77]:
def min_pivot(xs, n):
    coeffs = []
    for _ in range(n):
        l = None
        for i, xi in enumerate(xs):
            if (l is None or frac(xs[l]) > frac(xi)) and frac(xi) != 0:
                l = i
        if l is None:
            break
            
        xs, c = pivot(xs, l)
        coeffs += c
    return xs, coeffs